In [2]:
# 라이브러리 import
import numpy as np
import pandas as pd
import mariadb
import sys
from sklearn.feature_extraction.text import CountVectorizer  #텍스트(String) 벡터화
from sklearn.metrics.pairwise import cosine_similarity  #코사인 유사도
import pickle
import gzip


# DB Contents 데이터 불러오기

In [3]:
conn = mariadb.connect(
    user="eoeo0326",
    password="2017150009",
    host="oosoords.cqo7bp08tx6h.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    db="oosooDB"
)

contents_query = "SELECT * FROM contents"

contents_df = pd.read_sql(contents_query, conn)

contents_df.sort_values(by='vote_average', ascending=False).head(30)

d:\programming\python\python\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,_type,title,genre,production_countries,vote_count,vote_average,number_of_seasons,number_of_episodes,release_date,adult,poster_path,runtime,overview,now_status,flatrate,rent,buy,link
17922,m_552482,movie,Lowriders vs Zombies from Space,"공포,",,1,10.0,NaN,NaN,2018-09-30,0.0,/ucRKJ3DT6eOXW1jKZMUULeQpwuu.jpg,56.0,,None,,,,https://www.themoviedb.org/movie/552482-lowrid...
24987,m_760880,movie,Ya no estoy aquí: Una conversación entre Guill...,,,1,10.0,NaN,NaN,2020-10-30,0.0,/9BWxL6O64eRfEYIYL8fezu2A4F8.jpg,14.0,,None,"Netflix,",,,https://www.themoviedb.org/movie/760880-ya-no-...
24691,m_746542,movie,Nations United: Urgent Solutions for Urgent Times,"다큐멘터리,",,1,10.0,NaN,NaN,2020-09-19,0.0,/sZs69zqBRFw0zvHi9LFXuZEPtLs.jpg,34.0,,None,"GuideDoc,",,,https://www.themoviedb.org/movie/746542-nation...
24744,m_749069,movie,Iti Srikanta,"드라마,","India,",1,10.0,NaN,NaN,2004-06-04,0.0,None,125.0,,None,"Hoichoi,",,,https://www.themoviedb.org/movie/749069-iti-sr...
24748,m_749464,movie,Bloodhound,"코미디, 공포,",,1,10.0,NaN,NaN,2020-10-24,0.0,/ia5fiT47kCddcOUMqwiabKe1zuC.jpg,4.0,,None,"Watcha,",,"Naver Store,",https://www.themoviedb.org/movie/749464-bloodh...
24789,m_751494,movie,Manne No 13,"공포, 스릴러,","India,",1,10.0,NaN,NaN,2020-11-26,0.0,/eKWNp7rHsRdhBJ3jp1knPH93WYQ.jpg,105.0,,None,"Amazon Prime Video,",,,https://www.themoviedb.org/movie/751494-manne-...
24790,m_751545,movie,와일드 포르투갈,"다큐멘터리, TV 영화,","Germany,",1,10.0,NaN,NaN,2020-10-09,0.0,/8qkAGh89JWXW5BtiZGBHOOCNikd.jpg,44.0,"바위로 뒤덮인 해안가, 고립된 산꼭대기, 황금빛 초원을 머릿속에 그려보자. 여기에 ...",None,"Disney Plus,",,,https://www.themoviedb.org/movie/751545-wild-p...
24822,m_752885,movie,바다 탐험대 옥토넛: 대산호초 보호작전,"가족, 음악, 애니메이션,",,1,10.0,NaN,NaN,2020-11-12,0.0,/lySPdyRyz4hfgNEKxFPNsgWiOci.jpg,45.0,아름다운 산호초로 뒤덮인 호주 대산호초에는 다양한 바다 생물들이 함께 모여 살아가...,None,"Netflix,",,"Naver Store,",https://www.themoviedb.org/movie/752885-octona...
24853,m_755033,movie,Life After The Navigator,"다큐멘터리,",,1,10.0,NaN,NaN,2020-11-09,0.0,/ja4kt09necDqk4whIDfTWi2cujl.jpg,90.0,,None,,,,https://www.themoviedb.org/movie/755033-life-a...
24893,m_756851,movie,Zver,"음악,","United Kingdom,",1,10.0,NaN,NaN,2020-11-03,0.0,None,5.0,,None,"wavve, Watcha,","wavve,","Naver Store,",https://www.themoviedb.org/movie/756851-zver/w...


In [4]:
# vote_count가 적은 작품들이 vote_average 상위에 노출되는것을 알 수 있음
# 이를 해결하기 위해 IMDB의 weighted_average 사용

# IMDB Weighted Vote Average

In [5]:
# IMDB의 가중 평점 계산법
# W = (vR + mC) / (v + m)

# W: 가중 평점
# v: 해당 영화의 총 투표수
# m: 최소한의 투표 수
# R: 해당 영화의 평균 평점
# C: 모든 영화의 평균 평점

percentile = 0.9  # vote_count 상위 몇 % 를 기준으로 삼을 것인지, 0.9 = 상위 10%
m = contents_df['vote_count'].quantile(percentile)
C = contents_df['vote_average'].mean()

def weighted_vote_average(contents):
    v = contents['vote_count']
    R = contents['vote_average']
    
    return ((v * R) + (m * C)) / (v + m)

contents_df['weighted_average'] = contents_df.apply(weighted_vote_average, axis=1)

contents_df.sort_values(by='weighted_average', ascending=False).head(30)

,id,_type,title,genre,production_countries,vote_count,vote_average,number_of_seasons,number_of_episodes,release_date,adult,poster_path,runtime,overview,now_status,flatrate,rent,buy,link,weighted_average
6594,m_278,movie,쇼생크 탈출,"드라마, 범죄,","United States of America,",20723,8.7,NaN,NaN,1994-09-23,0.0,/oAt6OtpwYCdJI76AVtVKW1eorYx.jpg,142.0,촉망받는 은행 간부 앤디 듀프레인은 아내와 그녀의 정부를 살해했다는 누명을 쓴다. ...,None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",https://www.themoviedb.org/movie/278-the-shaws...,8.573333
5077,m_238,movie,대부,"드라마, 범죄,","United States of America,",15465,8.7,NaN,NaN,1972-03-14,0.0,/cOwVs8eYA4G9ZQs7hIRSoiZr46Q.jpg,175.0,"시실리에서 이민온 뒤, 정치권까지 영향력을 미치는 거물로 자리잡은 돈 꼴레오네는 갖...",None,,"wavve,","wavve,",https://www.themoviedb.org/movie/238-the-godfa...,8.532190
30274,t_1396,tv,브레이킹 배드,"드라마,","United States of America,",8262,8.8,5.0,62.0,2008-01-20,NaN,/ztkUQFLlC19CCMYHW9o1zWhJRNq.jpg,NaN,2008년 1월 AMC에서 방영을 시작한 범죄 스릴러. Breaking Bad는 막...,Ended,"Netflix,",,,https://www.themoviedb.org/tv/1396-breaking-ba...,8.489615
2463,m_155,movie,다크 나이트,"드라마, 액션, 범죄, 스릴러,","United Kingdom, United States of America,",26725,8.5,NaN,NaN,2008-07-14,0.0,/f6dNinWX8rBM79JXKcShkfSh2oA.jpg,152.0,범죄와 부정부패를 제거하여 고담시를 지키려는 배트맨. 그는 짐 고든 형사와 패기 넘...,None,"wavve,","wavve, Naver Store,","wavve, Naver Store,",https://www.themoviedb.org/movie/155-the-dark-...,8.406299
32617,t_69050,tv,리버데일,"범죄, 드라마, 미스터리,","United States of America,",12497,8.6,6.0,102.0,2017-01-26,NaN,/6zBWSuYW3Ps1nTfeMS8siS4KUaA.jpg,NaN,"평화로운 소도시 리버데일에서 살아가는 아치와 친구들이 복잡한 연애, 섹스, 학교생활...",Returning Series,"Netflix,",,,https://www.themoviedb.org/tv/69050-riverdale/...,8.399984
12147,m_424,movie,쉰들러 리스트,"드라마, 역사, 전쟁,","United States of America,",12366,8.6,NaN,NaN,1993-11-30,0.0,/oyyUcGwLX7LTFS1pQbLrQpyzIyt.jpg,195.0,2차 세계대전 당시 독일군이 점령한 폴란드. 시류에 맞춰 자신의 성공을 추구하는 기...,None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",https://www.themoviedb.org/movie/424-schindler...,8.397983
31756,t_60625,tv,릭 앤 모티,"애니메이션, 코미디, Sci-Fi & Fantasy, Action & Adventu...","United States of America,",6170,8.8,5.0,51.0,2013-12-02,NaN,/8kOWDBK6XlPUzckuHDo3wwVRFwt.jpg,NaN,술에 절어 사는 천재 과학자 릭은 투정이 심한 10대 손자 모티를 데리고 외계와 대...,Returning Series,"Netflix,",,,https://www.themoviedb.org/tv/60625-rick-and-m...,8.395415
23041,m_680,movie,펄프 픽션,"스릴러, 범죄,","United States of America,",22599,8.5,NaN,NaN,1994-09-10,0.0,/6lXRHGoEbnnBUKsuqpL9JxD4DzT.jpg,154.0,펌프킨와 허니 버니가 레스토랑에서 강도 행각을 벌이기 시작한다. 빈센트와 그 동료 ...,None,"wavve,",,,https://www.themoviedb.org/movie/680-pulp-fict...,8.389722
1478,m_13,movie,포레스트 검프,"코미디, 드라마, 로맨스,","United States of America,",22068,8.5,NaN,NaN,1994-07-06,0.0,/xdJxoq0dtkchOkUz5UVKuxn7a2V.jpg,142.0,"불편한 다리, 남들보다 조금 떨어지는 지능을 가진 포레스트 검프는 헌신적인 어머니의...",None,"Netflix, wavve, Watcha,","wavve, Naver Store,","wavve, Naver Store,",https://www.themoviedb.org/movie/13-forrest-gu...,8.387153
33170,t_75006,tv,엄브렐러 아카데미,"Action & Adventure, Sci-Fi & Fantasy, 드라마,","Canada, United States of America,",7676,8.7,3.0,30.0,2019-02-15,NaN,/UBhjjSojL8z2ThtitnyWaKM8nq.jpg,NaN,초능력을 가진 남매들이 아버지의 장례식에서 다시 모인다. 그들 앞에 드러나는 가족의...,Returning Series,"Netflix,",,,https://www.themoviedb.org/tv/75006-the-umbrel...,8.376568


# 유사한 영화 추천 함수

In [6]:
# 입력한 영화와 장르가 유사한 영화를 반환하는 함수
def find_similar_contents(contents, similarity, contents_id, top_n=10):
    
    # 입력한 id를 가진 영화의 index 구하기 (similarity 행렬의 몇번째 인덱스인지)
    id_content = contents[contents['id'] == contents_id]
    content_index = id_content.index.values
    
    # 입력한 영화의 장르 유사도를 contents 데이터 프레임에 추가
    contents["similarity"] = similarity[content_index, :].reshape(-1,1)
    
    # 장르 유사도와 가중 평점을 기준으로 내림차순 정렬 후 상위 index 추출
    sorted_contents = contents.sort_values(by=['similarity', 'weighted_average'], ascending=False)
    sorted_contents = sorted_contents[sorted_contents.index.values != content_index]
    result = sorted_contents.index.values[ : top_n]
    
    return contents.iloc[result]

# 장르 유사도 측정

In [7]:
# CountVectorizer
count_vector = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_matrix = count_vector.fit_transform(contents_df['genre'])

genre_matrix

<35423x472 sparse matrix of type '<class 'numpy.int64'>'
	with 97596 stored elements in Compressed Sparse Row format>

In [8]:
# Cosine Similarity
genre_similarity = cosine_similarity(genre_matrix, genre_matrix)

# Save Pickle, gzip으로 압축하여 저장
# with gzip.open("genre_sim.pickle", "wb") as fw:
#     pickle.dump(genre_similarity, fw)

genre_similarity[0]

array([1.       , 0.       , 0.1490712, ..., 0.       , 0.       ,
       0.       ])

In [9]:
# 영화 '대부' 로 장르 유사도 기반 테스트
similar_genre_contents1 = find_similar_contents(contents_df, genre_similarity, 'm_238', top_n=20)
similar_genre_contents1[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
6594,m_278,쇼생크 탈출,"드라마, 범죄,",1994-09-23,8.573333,1.0
5129,m_240,대부 2,"드라마, 범죄,",1974-12-20,8.336685,1.0
25142,m_769,좋은 친구들,"드라마, 범죄,",1990-09-12,8.255336,1.0
31751,t_60574,피키 블라인더스,"드라마, 범죄,",2013-09-12,8.187165,1.0
7834,m_311,원스 어폰 어 타임 인 아메리카,"드라마, 범죄,",1984-05-23,7.871430,1.0
21276,m_640,캐치 미 이프 유 캔,"드라마, 범죄,",2002-12-25,7.827997,1.0
20761,m_627,트레인스포팅,"드라마, 범죄,",1996-02-23,7.745843,1.0
15584,m_4982,아메리칸 갱스터,"드라마, 범죄,",2007-11-02,7.210698,1.0
33940,t_82883,디 액트,"드라마, 범죄,",2019-03-20,7.148897,1.0
32492,t_67744,마인드헌터,"드라마, 범죄,",2017-10-13,6.986052,1.0


In [10]:
# 영화 '아이언맨' 으로 장르 유사도 기반 테스트
similar_genre_contents2 = find_similar_contents(contents_df, genre_similarity, 'm_1726', top_n=20)
similar_genre_contents2[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
6340,m_27205,인셉션,"액션, SF, 모험,",2010-07-15,8.320974,1.000000
1026,m_118340,가디언즈 오브 갤럭시,"액션, SF, 모험,",2014-07-30,7.813282,1.000000
19257,m_588228,투모로우 워,"액션, SF, 모험,",2021-07-02,7.626684,1.000000
15473,m_49538,엑스맨: 퍼스트 클래스,"액션, SF, 모험,",2011-06-01,7.152518,1.000000
11340,m_399579,알리타: 배틀 엔젤,"액션, SF, 모험,",2019-01-31,6.996635,1.000000
23230,m_68726,퍼시픽 림,"액션, SF, 모험,",2013-07-11,6.678633,1.000000
10711,m_38356,트랜스포머: 달의 어둠,"액션, SF, 모험,",2011-06-28,5.988662,1.000000
13838,m_458478,카르텔 2045,"액션, SF, 모험,",2017-04-01,4.929909,1.000000
3438,m_185471,배틀 오브 머신,"액션, SF, 모험,",2013-07-25,4.927399,1.000000
13111,m_443297,문트랩: 타겟 어스,"액션, SF, 모험,",2017-02-07,4.920561,1.000000


In [11]:
gnt_similarity = genre_similarity

In [12]:
del genre_similarity

In [13]:
del genre_matrix

# 제목 유사도 측정

In [14]:
# CountVectorizer
title_matrix = count_vector.fit_transform(contents_df['title'])

title_matrix

<35423x77651 sparse matrix of type '<class 'numpy.int64'>'
	with 134479 stored elements in Compressed Sparse Row format>

In [15]:
# Cosine Similarity
title_similarity = cosine_similarity(title_matrix, title_matrix)

# Save Pickle, gzip으로 압축하여 저장
# with gzip.open("title_sim.pickle", "wb") as fw:
#     pickle.dump(title_similarity, fw)

title_similarity[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [16]:
# 영화 '대부' 로 제목 유사도 기반 테스트
similar_title_contents1 = find_similar_contents(contents_df, title_similarity, 'm_238', top_n=20)
similar_title_contents1[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
5129,m_240,대부 2,"드라마, 범죄,",1974-12-20,8.336685,1.0
5191,m_242,대부 3,"범죄, 드라마, 스릴러,",1990-12-25,7.074674,1.0
6594,m_278,쇼생크 탈출,"드라마, 범죄,",1994-09-23,8.573333,0.0
30274,t_1396,브레이킹 배드,"드라마,",2008-01-20,8.489615,0.0
2463,m_155,다크 나이트,"드라마, 액션, 범죄, 스릴러,",2008-07-14,8.406299,0.0
32617,t_69050,리버데일,"범죄, 드라마, 미스터리,",2017-01-26,8.399984,0.0
12147,m_424,쉰들러 리스트,"드라마, 역사, 전쟁,",1993-11-30,8.397983,0.0
31756,t_60625,릭 앤 모티,"애니메이션, 코미디, Sci-Fi & Fantasy, Action & Adventu...",2013-12-02,8.395415,0.0
23041,m_680,펄프 픽션,"스릴러, 범죄,",1994-09-10,8.389722,0.0
1478,m_13,포레스트 검프,"코미디, 드라마, 로맨스,",1994-07-06,8.387153,0.0


In [17]:
# 영화 '아이언맨' 으로 제목 유사도 기반 테스트
similar_title_contents2 = find_similar_contents(contents_df, title_similarity, 'm_1726', top_n=20)
similar_title_contents2[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
23227,m_68721,아이언맨 3,"액션, 모험, SF,",2013-04-18,6.828512,1.000000
90,m_10138,아이언맨 2,"모험, 액션, SF,",2010-04-28,6.726187,1.000000
31890,t_61739,아이언맨,"드라마, Sci-Fi & Fantasy,",2014-09-10,4.953826,1.000000
4838,m_230896,아이언맨 & 헐크,"액션, 모험, 애니메이션, 가족,",2013-12-03,5.068504,0.577350
6811,m_284274,아이언맨 & 캡틴아메리카 히어로즈 유나이티드,"모험, 애니메이션, 액션,",2014-07-29,5.116277,0.377964
2971,m_169934,아이언맨: 라이즈 오브 테크노보어,"SF, 애니메이션, 액션,",2013-04-24,5.073380,0.377964
21100,m_635802,일론 머스크: 리얼 아이언맨,"다큐멘터리,",2018-12-04,4.947316,0.377964
6594,m_278,쇼생크 탈출,"드라마, 범죄,",1994-09-23,8.573333,0.000000
5077,m_238,대부,"드라마, 범죄,",1972-03-14,8.532190,0.000000
30274,t_1396,브레이킹 배드,"드라마,",2008-01-20,8.489615,0.000000


In [18]:
gnt_similarity += title_similarity

In [19]:
del title_similarity

In [20]:
del title_matrix

# 장르 유사도 + 제목 유사도 파일 저장

In [21]:
# Save Pickle, gzip으로 압축하여 저장
# with gzip.open("gnt_sim.pickle", "wb") as fw:
#     pickle.dump(gnt_similarity, fw)

gnt_similarity[0]

array([2.       , 0.       , 0.1490712, ..., 0.       , 0.       ,
       0.       ])

In [23]:
# 영화 '대부' 로 장르, 제목 유사도 기반 테스트
similar_genre_contents1 = find_similar_contents(contents_df, gnt_similarity, 'm_238', top_n=20)
similar_genre_contents1[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
5129,m_240,대부 2,"드라마, 범죄,",1974-12-20,8.336685,2.000000
5191,m_242,대부 3,"범죄, 드라마, 스릴러,",1990-12-25,7.074674,1.516398
6594,m_278,쇼생크 탈출,"드라마, 범죄,",1994-09-23,8.573333,1.000000
25142,m_769,좋은 친구들,"드라마, 범죄,",1990-09-12,8.255336,1.000000
31751,t_60574,피키 블라인더스,"드라마, 범죄,",2013-09-12,8.187165,1.000000
7834,m_311,원스 어폰 어 타임 인 아메리카,"드라마, 범죄,",1984-05-23,7.871430,1.000000
21276,m_640,캐치 미 이프 유 캔,"드라마, 범죄,",2002-12-25,7.827997,1.000000
20761,m_627,트레인스포팅,"드라마, 범죄,",1996-02-23,7.745843,1.000000
15584,m_4982,아메리칸 갱스터,"드라마, 범죄,",2007-11-02,7.210698,1.000000
33940,t_82883,디 액트,"드라마, 범죄,",2019-03-20,7.148897,1.000000


In [24]:
# 영화 '아이언맨' 으로 장르, 제목 유사도 기반 테스트
similar_genre_contents2 = find_similar_contents(contents_df, gnt_similarity, 'm_1726', top_n=20)
similar_genre_contents2[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
90,m_10138,아이언맨 2,"모험, 액션, SF,",2010-04-28,6.726187,1.800000
23227,m_68721,아이언맨 3,"액션, 모험, SF,",2013-04-18,6.828512,1.600000
31890,t_61739,아이언맨,"드라마, Sci-Fi & Fantasy,",2014-09-10,4.953826,1.000000
6340,m_27205,인셉션,"액션, SF, 모험,",2010-07-15,8.320974,1.000000
1026,m_118340,가디언즈 오브 갤럭시,"액션, SF, 모험,",2014-07-30,7.813282,1.000000
19257,m_588228,투모로우 워,"액션, SF, 모험,",2021-07-02,7.626684,1.000000
15473,m_49538,엑스맨: 퍼스트 클래스,"액션, SF, 모험,",2011-06-01,7.152518,1.000000
11340,m_399579,알리타: 배틀 엔젤,"액션, SF, 모험,",2019-01-31,6.996635,1.000000
23230,m_68726,퍼시픽 림,"액션, SF, 모험,",2013-07-11,6.678633,1.000000
10711,m_38356,트랜스포머: 달의 어둠,"액션, SF, 모험,",2011-06-28,5.988662,1.000000


In [21]:
del gnt_similarity

# Overview 유사도 측정 (-> 추천 척도로 적합하지 않다고 판단됨)

In [30]:
# CountVectorizer
overview_matrix = count_vector.fit_transform(contents_df['overview'])

overview_matrix

<35396x1179421 sparse matrix of type '<class 'numpy.int64'>'
	with 2234162 stored elements in Compressed Sparse Row format>

In [31]:
# Cosine Similarity
overview_similarity = cosine_similarity(overview_matrix, overview_matrix)

overview_similarity[0]

array([1.        , 0.        , 0.01145291, ..., 0.        , 0.00909429,
       0.        ])

In [32]:
# 영화 '대부' 로 제목 유사도 기반 테스트
similar_overview_contents1 = find_similar_contents(contents_df, overview_similarity, 'm_238', top_n=20)
similar_overview_contents1[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
18249,m_561974,점박이 한반도의 공룡 2: 새로운 낙원,"애니메이션, 모험,",2019-12-25,5.278670,0.088549
5364,m_24833,산타 버디즈,"가족, 모험,",2009-10-22,5.126027,0.088283
14205,m_466081,비밀정보원: 인 더 프리즌,"드라마, 범죄, 스릴러,",2019-08-30,5.621045,0.087118
20235,m_612701,"프랑켄슈타인의 괴물의 괴물, 프랑켄슈타인","코미디,",2019-07-16,5.019687,0.081923
14997,m_484889,라이즈 오브 더 풋솔져 3,"액션, 범죄, 드라마, 스릴러,",2017-11-03,4.993888,0.080301
34449,t_88775,양키,"범죄, 드라마,",2019-06-14,4.960981,0.074959
1644,m_134411,스니치,"스릴러, 드라마, 액션,",2013-02-21,5.719189,0.074476
29608,t_130523,더 크루: 갱랜드,"범죄, Action & Adventure,",2021-09-24,5.182359,0.071698
32891,t_72339,해피,"Sci-Fi & Fantasy, 코미디,",2017-12-06,5.613193,0.069238
11543,m_412361,니트로 러쉬,"액션,",2016-08-31,4.967003,0.067028


In [33]:
# 영화 '아이언맨' 으로 제목 유사도 기반 테스트
similar_overview_contents2 = find_similar_contents(contents_df, overview_similarity, 'm_1726', top_n=20)
similar_overview_contents2[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
90,m_10138,아이언맨 2,"모험, 액션, SF,",2010-04-28,6.726282,0.196145
34300,t_87194,스페셜,"코미디,",2019-04-12,5.002301,0.099945
28913,t_1154,초전자 바이오맨,"Sci-Fi & Fantasy, Action & Adventure,",1984-02-04,4.948022,0.094597
31572,t_52883,전용,"Action & Adventure, 애니메이션, 코미디,",2013-01-09,4.955441,0.090886
856,m_11485,돈을 갖고 튀어라,"코미디, 범죄,",1969-08-18,5.964000,0.083071
20500,m_619263,더 웨이브,"SF,",2019-09-21,5.035940,0.081517
13391,m_449985,트리플 스렛,"스릴러, 액션,",2019-03-19,5.510834,0.078899
13014,m_44115,127 시간,"모험, 드라마, 스릴러,",2010-11-12,6.876409,0.077559
3711,m_19426,카비리아의 밤,"드라마,",1957-10-03,6.273752,0.076991
12312,m_42738,스콜피오,"액션, 드라마, 스릴러,",1973-04-11,5.068502,0.076064


In [34]:
del overview_similarity

In [35]:
del overview_matrix

# Release Date 유사도 측정 (-> 추천 척도로 적합하지 않다고 판단됨)

In [21]:
# release_date to str
dates = pd.to_datetime(contents_df['release_date'])
release_date = dates.dt.strftime('%Y-%m-%d')

type(release_date)

pandas.core.series.Series

In [23]:
release_date = release_date.fillna('0')

In [24]:
# CountVectorizer
date_matrix = count_vector.fit_transform(release_date)

date_matrix

<35396x1661 sparse matrix of type '<class 'numpy.int64'>'
	with 171320 stored elements in Compressed Sparse Row format>

In [25]:
# Cosine Similarity
date_similarity = cosine_similarity(date_matrix, date_matrix)

date_similarity[0]

array([1. , 0. , 0. , ..., 0.2, 0. , 0. ])

In [26]:
# 영화 '대부' 로 Release Date 유사도 기반 테스트
similar_date_contents1 = find_similar_contents(contents_df, date_similarity, 'm_238', top_n=20)
similar_date_contents1[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
17045,m_527641,파이브 피트,"로맨스, 드라마,",2019-03-14,7.850318,0.6
3776,m_197,브레이브하트,"액션, 드라마, 역사, 전쟁,",1995-03-14,7.657587,0.6
30636,t_16997,퍼시픽,"드라마, Action & Adventure, War & Politics,",2010-03-14,6.909610,0.6
26615,m_853,에너미 앳 더 게이트,"드라마, 전쟁, 역사,",2001-03-14,6.886583,0.6
2539,m_157350,다이버전트,"액션, 모험, SF,",2014-03-14,6.780100,0.6
13846,m_458723,어스,"공포, 스릴러, 미스터리,",2019-03-14,6.766071,0.6
19445,m_593,솔라리스,"드라마, SF, 미스터리,",1972-03-20,6.765436,0.6
21505,m_64688,21 점프 스트리트,"액션, 코미디, 범죄,",2012-03-14,6.750134,0.6
14630,m_476299,베스와 베라,"공포, 미스터리, 스릴러,",2018-03-14,6.700952,0.6
11746,m_416144,호텔 뭄바이,"스릴러, 역사, 드라마, 액션,",2019-03-14,6.568020,0.6


In [27]:
# 영화 '아이언맨' 으로 Release Date 유사도 기반 테스트
similar_date_contents2 = find_similar_contents(contents_df, date_similarity, 'm_1726', top_n=20)
similar_date_contents2[['id', 'title', 'genre', 'release_date', 'weighted_average', 'similarity']]

,id,title,genre,release_date,weighted_average,similarity
453,m_10761,남주기 아까운 그녀,"코미디, 로맨스,",2008-04-30,5.895015,1.0
12141,m_42384,가루지기,"코미디, 판타지, 드라마,",2008-04-30,4.981937,1.0
23379,m_69383,비스티 보이즈,"코미디, 드라마,",2008-04-30,4.951392,1.0
370,m_10625,퀸카로 살아남는 법,"코미디,",2004-04-30,6.989614,0.6
34229,t_86248,업로드,"코미디, Sci-Fi & Fantasy,",2020-04-30,6.582623,0.6
31440,t_46639,아메리칸 갓,"드라마, 미스터리, Sci-Fi & Fantasy,",2017-04-30,6.297329,0.6
22188,m_656563,리치 인 러브,"코미디, 로맨스,",2020-04-30,6.131379,0.6
3067,m_1729,포비든 킹덤: 전설의 마스터를 찾아서,"액션, 모험, 판타지,",2008-04-18,6.008560,0.6
1355,m_1266,스트리트 킹,"액션, 범죄, 드라마, 스릴러,",2008-04-10,5.931573,0.6
4631,m_22536,박쥐,"드라마, 공포, 스릴러,",2009-04-30,5.914320,0.6


In [28]:
del date_similarity

In [29]:
del date_matrix